# Multi-Agent Collaboration (MAC) Project : Hedge Fund Assistant
---

This example demonstrates a Hedge Fund Assistant using MAC principles with the following structure:

- A **Supervisor Agent** powered by the Amazon Nova Lite foundation model routes user queries.

- Three **Sub-Agents**:
    - **Fundamental Analyst Agent** (uses Claude 3 Haiku as the FM)
    - **Technical Analyst Agent** (uses Amazon Nova Lite as the FM)
    - **Market Analyst Agent** (uses Claude 3 Haiku as the FM)

Each sub-agent has access to a set of tools to fetch data, analyze it, and provide actionable insights.

#### Set a logger

The Strands Agents SDK implements a straightforward logging approach:

1. **Module-level Loggers**: Each module in the SDK creates its own logger using logging.getLogger(__name__), following Python best practices for hierarchical logging.

2. **Root Logger**: All loggers in the SDK are children of the "strands" root logger, making it easy to configure logging for the entire SDK.

3. **Default Behavior**: By default, the SDK doesn't configure any handlers or log levels, allowing you to integrate it with your application's logging configuration.

In [ ]:
# import logging and set a logger for strands
import logging
# import the strands agents and strands tools that we will be using
from strands import Agent
from strands import tool
from strands_tools import swarm
from typing import Optional, Dict, Any, List, Union


# configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

In [ ]:
# install other requirements
import os 
import sys 
import json 
import time
import boto3
import shutil 
import zipfile
import subprocess
import finnhub
import uuid
from datetime import datetime
import requests 
import tempfile 
from litellm import completion
from dotenv import load_dotenv
load_dotenv(override=True)    # This will read the .env file and add variables to os.environ


# load the environment variables that are defined in the ".env file" . This contains the 
# financial data API key that will enable the user to access the data 
os.environ["LANGFUSE_PUBLIC_KEY"]= os.getenv("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"]= os.getenv("LANGFUSE_SECRET_KEY")
os.environ["LANGFUSE_HOST"]= os.getenv("LANGFUSE_HOST")

### Load config file
---

This [config](config.yaml) file is a `yaml` file that contains information that this solution uses, including the result files, the model information for each of the agent used in this multi agentic system, inference parameters, and more.

In [3]:
!uv pip install PyYAML aiohttp openai
from utils import load_config

# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Load the config file
config_data = load_config('config.yaml')
logger.info(f"Loaded config from local file system: {json.dumps(config_data, indent=2)}")

Audited 3 packages in 564ms


INFO | utils | Loading config from local file system: config.yaml
INFO | utils | Loaded config from local file system: {'general': {'name': 'Financial-multi-agentic-system', 'description': 'This agentic system shows multi-agent collaboration different financial agents'}, 'model_information': {'finance_agent_model_info': {'model_id': 'us.amazon.nova-micro-v1:0', 'inference_parameters': {'temperature': 0.1}}, 'technical_agent_model_info': {'model_id': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0', 'inference_parameters': {'temperature': 0.1, 'max_tokens': 512}}, 'market_analysis_agent': {'model_id': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0', 'inference_parameters': {'temperature': 0.1, 'max_tokens': 512}}}, 'code_gen_model_info': {'model_id': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0', 'inference_parameters': {'temperature': 0.1, 'max_tokens': 2048}}}


### Define tools
---

This example creates a supervisor agent and integrates the three sub-agents, each having specific tools for financial data analysis:

1. **Fundamental Analyst Agent**:
    - Tools: Retrieve income statements, balance sheets, and cash flow statements.

2. **Technical Analyst Agent**:
    - Tools: Fetch stock prices, current prices, and compute technical indicators (e.g., `RSI`, `MACD`, `SMA`).

3. **Market Analyst Agent**:
    - Tools: Access options chain data, insider trading information, and market news.

In [4]:
api_key = os.environ.get("FINANCIAL_DATASET_API")
if api_key is not None:
    print("Financial Data API key found in environment variables.")
else:
    print(f"API key not found, enter it in the .env file.")

Financial Data API key found in environment variables.


In [ ]:
## dry run 
client = finnhub.Client(api_key=api_key)
quote = client.quote('AAPL')
print("Apple Quote:", quote)

Apple Quote: {'c': 201.895, 'd': 0.395, 'dp': 0.196, 'h': 203.08, 'l': 200.2, 'o': 203.08, 'pc': 201.5, 't': 1750780929}


In [6]:
tool_use_ids=[]

# ─── LOGGER SETUP ──────────────────────────────────────────────────────────────
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def comprehensive_callback_handler(**kwargs):
    """
    Enhanced comprehensive callback handler with LangSmith integration
    """
    
    # === REASONING EVENTS (Agent's thinking process) ===
    if kwargs.get("reasoning", False):
        if "reasoningText" in kwargs:
            reasoning_text = kwargs['reasoningText']
            logger.info(f"🧠 REASONING: {reasoning_text}")
            
        if "reasoning_signature" in kwargs:
            logger.info(f"🔍 REASONING SIGNATURE: {kwargs['reasoning_signature']}")
    
    # === TEXT GENERATION EVENTS ===
    elif "data" in kwargs:
        # Log streamed text chunks from the model
        logger.info(kwargs["data"], end="")
        if kwargs.get("complete", False):
            logger.info("")  # Add newline when complete
    
    # === TOOL EVENTS ===
    elif "current_tool_use" in kwargs:
        tool = kwargs["current_tool_use"]
        tool_use_id = tool["toolUseId"]
        
        if tool_use_id not in tool_use_ids:
            tool_name = tool.get('name', 'unknown_tool')
            tool_input = tool.get('input', {})
            
            logger.info(f"\n🔧 USING TOOL: {tool_name}")
            if "input" in tool:
                logger.info(f"📥 TOOL INPUT: {tool_input}")
            tool_use_ids.append(tool_use_id)
    
    # === TOOL RESULTS ===
    elif "tool_result" in kwargs:
        tool_result = kwargs["tool_result"]
        tool_use_id = tool_result.get("toolUseId")
        result_content = tool_result.get("content", [])
        
        logger.info(f"📤 TOOL RESULT: {result_content}")
    
    # === LIFECYCLE EVENTS ===
    elif kwargs.get("init_event_loop", False):
        logger.info("🔄 Event loop initialized")
        
    elif kwargs.get("start_event_loop", False):
        logger.info("▶️ Event loop cycle starting")
        
    elif kwargs.get("start", False):
        logger.info("📝 New cycle started")
        
    elif kwargs.get("complete", False):
        logger.info("✅ Cycle completed")
        
    elif kwargs.get("force_stop", False):
        reason = kwargs.get("force_stop_reason", "unknown reason")
        logger.info(f"🛑 Event loop force-stopped: {reason}")
    
    # === MESSAGE EVENTS ===
    elif "message" in kwargs:
        message = kwargs["message"]
        role = message.get("role", "unknown")
        logger.info(f"📬 New message created: {role}")
    
    # === ERROR EVENTS ===
    elif "error" in kwargs:
        error_info = kwargs["error"]
        logger.error(f"❌ ERROR: {error_info}")

    # === RAW EVENTS (for debugging) ===
    elif "event" in kwargs:
        # Log raw events from the model stream (optional, can be verbose)
        logger.debug(f"🔍 RAW EVENT: {kwargs['event']}")
    
    # === DELTA EVENTS ===
    elif "delta" in kwargs:
        # Raw delta content from the model
        logger.debug(f"📊 DELTA: {kwargs['delta']}")
    
    # === CATCH-ALL FOR DEBUGGING ===
    else:
        # Log any other events we might have missed
        logger.debug(f"❓ OTHER EVENT: {kwargs}")


In [ ]:
# define the tools for the finance agent
# Strands Agents SDK makes it straightforward to 
# turn Python functions into callable tools within an AI agent.

# transform any Python function into a tool simply by adding the @tool decorator—its 
# docstring and type hints automatically generate the tool's specification

## TOOL 1
@tool
def get_income_statements(
    ticker: str, 
    period: str = "ttm", 
    limit: int = 10
) -> Dict:
    """
    Get income statements for a ticker. This is one of the functions that is used to get the 
    income statements based on the ticker specified by the user
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        period (str, optional): Time period for the data. Options:
            - 'ttm': Trailing twelve months (default)
            - 'annual': Annual data
            - 'quarterly': Quarterly data
        limit (int, optional): Maximum number of statements to return (default: 10)
        
    Returns:
        Dict: JSON response containing income statement data
        
    Raises:
        RuntimeError: If API key is missing or API request fails
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = (
        f'https://api.financialdatasets.ai/financials/income-statements'
        f'?ticker={ticker}'
        f'&period={period}'
        f'&limit={limit}'
    )

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        print(f"OUTPUT FROM THE GET_INCOME_STATEMENTS TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        return {"ticker": ticker, "income_statements": [], "error": str(e)}

## TOOL 2
@tool
def get_balance_sheets(
    ticker: str, 
    period: str = "ttm", 
    limit: int = 10
) -> Dict:
    """
    Get balance sheets for a ticker and the specified limit and time period
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        period (str, optional): Time period for the data. Options:
            - 'ttm': Trailing twelve months (default)
            - 'annual': Annual data
            - 'quarterly': Quarterly data
        limit (int, optional): Maximum number of balance sheets to return (default: 10)
        
    Returns:
        Dict: JSON response containing balance sheet data
        
    Raises:
        RuntimeError: If API key is missing or API request fails
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = (
        f'https://api.financialdatasets.ai/financials/balance-sheets'
        f'?ticker={ticker}'
        f'&period={period}'
        f'&limit={limit}'
    )

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        print(f"OUTPUT FROM THE GET_BALANCE_SHEETS TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        return {"ticker": ticker, "balance_sheets": [], "error": str(e)}

## TOOL 3
@tool
def get_cash_flow_statements(
    ticker: str, 
    period: str = "ttm", 
    limit: int = 10
) -> Dict:
    """
    Get cash flow statements for a ticker
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        period (str, optional): Time period for the data. Options:
            - 'ttm': Trailing twelve months (default)
            - 'annual': Annual data
            - 'quarterly': Quarterly data
        limit (int, optional): Maximum number of cash flow statements to return (default: 10)
        
    Returns:
        Dict: JSON response containing cash flow data
        
    Raises:
        RuntimeError: If API key is missing or API request fails
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = (
        f'https://api.financialdatasets.ai/financials/cash-flow-statements'
        f'?ticker={ticker}'
        f'&period={period}'
        f'&limit={limit}'
    )

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        # fetch and return the output of the api call in the response
        print(f"OUTPUT FROM THE GET_CASH_FLOW_STATEMENTS TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        return {"ticker": ticker, "cash_flow_statements": [], "error": str(e)}

#### Create a custom python code execution tool
---

In this section, we will create a custom tool that lets you configure or use a model of choice to generate code based on the content and the user question and then execute that tool as well.

In [8]:
# Represent the code execution prompt for the LLM
CODE_EXEC_PROMPT : str = """You are a Python code generator. Generate clean, efficient, and safe Python code based on the user's requirements.

Guidelines:
1. Only generate executable Python code
2. Include necessary imports at the top
3. Add comments to explain complex logic 
4. Handle potential errors gracefully
5. Keep code concise but readable
6. Do not include any harmful or system-modifying operations
7. Focus on data processing, calculations, and analysis tasks
8. If working with data, assume it might be passed as context 
9. Print results using print() statements so they appear in output

Always adhere to the user question and generate python code based on what the user is asking.

Return ONLY the python code, no explainations or markdown formatting."""

In [9]:
! uv pip install tiktoken tokenizer jinja2

Audited 3 packages in 418ms


In [10]:
! uv pip show litellm

Name: litellm
Version: 1.72.7
Location: /mnt/c/code/Projects/Agentic_AI_AutoGen/AWSBedrock-Project-strands-multi-agentic-system/.venv/lib/python3.12/site-packages
Requires: aiohttp, click, httpx, importlib-metadata, jinja2, jsonschema, openai, pydantic, python-dotenv, tiktoken, tokenizers
Required-by: financial-analysis-multi-agent-collaboration


### Define the system prompt for the financial agent
---

In [13]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [14]:
# This is the system prompt that will be used for the financial agent
financial_agent_instruction = """You are a comprehensive fundamental analyst assistant that helps users analyze company financial statements across three key areas: income statements, balance sheets, and cash flow statements.
You require the user to provide a stock ticker symbol to analyze the company's financials.

You can perform the following types of analysis by calling the functions below:

1. Income Statement Analysis:
   - Revenue growth, gross profit, operating income, net income, earnings per share, and revenue/expense breakdown

2. Balance Sheet Analysis:
   - Asset composition (current and non-current), liabilities (current and non-current), shareholders' equity, retained earnings, and financial ratios (current ratio, debt-to-equity)

3. Cash Flow Statement Analysis:
   - Net cash flow from operations, capital expenditures, business acquisitions, issuance/repayment of debt, dividends, and changes in cash and equivalents

IMPORTANT: Always use the financial dataset API you have access to call these functions and retrieve the data to answer the user question

If you do not have access to the data that the user is asking for, do not make up an answer, just say that you do not know the answer. Be completely
accurate. Do not provide answers to anything but on the topic specified above. Always give information on where you got the data to answer the user question
and do not redact anything in your response.

Also, use the python repl tool function at the very end to generate report and analysis based on the context fetched so far and store
that in a file or provide that as an output.
"""
print(f"Going to use the financial agent system prompt: {financial_agent_instruction}")

Going to use the financial agent system prompt: You are a comprehensive fundamental analyst assistant that helps users analyze company financial statements across three key areas: income statements, balance sheets, and cash flow statements.
You require the user to provide a stock ticker symbol to analyze the company's financials.

You can perform the following types of analysis by calling the functions below:

1. Income Statement Analysis:
   - Revenue growth, gross profit, operating income, net income, earnings per share, and revenue/expense breakdown

2. Balance Sheet Analysis:
   - Asset composition (current and non-current), liabilities (current and non-current), shareholders' equity, retained earnings, and financial ratios (current ratio, debt-to-equity)

3. Cash Flow Statement Analysis:
   - Net cash flow from operations, capital expenditures, business acquisitions, issuance/repayment of debt, dividends, and changes in cash and equivalents

IMPORTANT: Always use the financial dat

In [15]:
# get the model information for the finance agent 
finance_agent_model_info: Dict = config_data['model_information'].get('finance_agent_model_info')
print(f"Fetched the agent model information: {finance_agent_model_info}")

Fetched the agent model information: {'model_id': 'us.amazon.nova-micro-v1:0', 'inference_parameters': {'temperature': 0.1}}


In [16]:
# initialize the model that will power the financial agent
# in this case, we will use the claude 3-7 model to power the financial agent
from strands.models import BedrockModel

# define the current aws region
region :str = boto3.Session().region_name
print(f"Going to use the agent in the region: {region}")

# create a bedrock model using the BedrockModel interface
bedrock_model = BedrockModel(
    model_id=finance_agent_model_info.get('model_id'),
    region_name = region,
    temperature= finance_agent_model_info['inference_parameters'].get('temperature')
)
print(f"Initialized the bedrock model for the finance agent: {bedrock_model}")

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.amazon.nova-micro-v1:0', 'temperature': 0.1}> | initializing


Going to use the agent in the region: us-east-1
Initialized the bedrock model for the finance agent: <strands.models.bedrock.BedrockModel object at 0x7fc2aa716090>


### Create the financial agent
---

Next, we will simply create the finance agent. We will use a Callback handler function in this case. We will implement the custom callback function. This will be invoked at various points throughout the financial agent's lifecycle. 

Here is an example that captures streamed data from the agent and logs instead of printing:

### What are Callback handlers in Strands SDK?

Callback handlers are a powerful feature for Strands SDK that enables the user to intercept and process the events as they are occurring during the agent execution process. This means that the agent will be able to handle real time monitoring, custom output formatting and integration with other external systems.

This handling is done and can occur throughout the agent's lifecycle and includes:

1. The text generation process that is done with the model that underlies the agent. If the agent is generating text then the callback handler will catch that and emit it out during the agent execution process.

2. If the agent is using tools or has tools associated with it, then the agent will be using that and executing that. As a part of that process, the callback handler can catch those traces and emit it out for real time processing.

3. Reasoning process: If you want to handle the traces or track the reasoning process of the agent based on the LLM thinking to debug and optimize performance, that is possible through callback handlers as well.

4. Errors and completions: If there are any errors or completions during the process we can handle that using the callback handler functionality as well.

For more information on what you can log with Callback handler events, view the documentation [here](https://strandsagents.com/0.1.x/user-guide/concepts/streaming/callback-handlers/).

In [17]:
# we will now set up logging for the agent using langfuse
import base64
otel_host = os.environ.get('LANGFUSE_HOST')
if otel_host:
    # Set up endpoint for OpenTelemetry
    otel_endpoint = str(os.environ.get("LANGFUSE_HOST")) + "/api/public/otel/v1/traces"

    # Create authentication token for OpenTelemetry
    auth_token = base64.b64encode(
        f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
    ).decode()
    os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = otel_endpoint
    os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {auth_token}"

In [18]:
# ! pip install "strands-agents[otel]"

In [19]:
# import the prebuilt tools.
# Calculator: Perform mathematical operations
# file_read: Read and parse files
# mem0_memory: Tool for managing memories using Mem0 (store, delete, list, get, and retrieve)
# python_repl: Run Python code
# current_time: Get the current date and time
# journal: Create structured tasks and logs for agents to manage and work from
from strands_tools import calculator, file_read, mem0_memory, python_repl, current_time, journal, python_repl

# Generate a unique session ID using UUID
session_id = str(uuid.uuid4())

# Create the financial agent
finance_agent = Agent(
    # this is the system prompt to the strands agent
    system_prompt=financial_agent_instruction, 
    # use the tools from the ones defined above for the 
    # finance agent
    tools = [
             # These are the custom built tools that the finance 
             # agent will use
             get_income_statements, 
             get_balance_sheets,
             get_cash_flow_statements, 
             # These are the prebuilt set of tools that the 
             # strands sdk agent already offers
             calculator, 
             python_repl, 
             file_read, 
             mem0_memory, 
             current_time, 
             journal],
    # define the callback handler in this case
    # This callback handler logs the reasoning, tool, lifecycle, 
    # raw, delta and message events.
    callback_handler=comprehensive_callback_handler, 
    trace_attributes={
            "session.id": session_id,  # Use UUID for unique session tracking
            "user.id": "agent-builder@strandsagents.com",
            "langfuse.tags": [
                "Strands-Agents-Builder",
            ],
        }
)

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}> | initializing
DEBUG | strands.tools.registry | tool_name=<get_income_statements> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_income_statements>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<get_balance_sheets> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_balance_sheets>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<get_cash_flow_statements> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_cash_flow_statements>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<calculator>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.loader | tool_path=</mnt/c/code/Projects/Agentic_AI_AutoGen/AWSBedrock-Pro

In [20]:
# view some of the tools and the model through the agent config 
finance_agent.model

In [23]:
# view the details of the tools in a more human readable manner
def print_tools_summary(config):
    """Print a summary of all tools"""
    tools = config.get('tools', [])
    
    print("=" * 60)
    print(f"AGENT TOOLS SUMMARY")
    print("=" * 60)
    print(f"Total number of tools: {len(tools)}")
    print()
    
    for i, tool in enumerate(tools, 1):
        tool_spec = tool.get('toolSpec', {})
        name = tool_spec.get('name', 'Unknown')
        description = tool_spec.get('description', 'No description available')
        
        # Extract the first line of description for summary
        first_line = description.split('\n')[0] if description else 'No description'
        
        print(f"{i}. {name}")
        print(f"   Summary: {first_line}")
        print()

In [24]:
# view the tools that the finance agent has access to 
print_tools_summary(finance_agent.tool_config)

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<get_income_statements> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_balance_sheets> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_cash_flow_statements> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<calculator> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<python_repl> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<file_read> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<mem0_memory> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<current_time> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<journal> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<9> | tools configured


AGENT TOOLS SUMMARY
Total number of tools: 9

1. get_income_statements
   Summary: Get income statements for a ticker. This is one of the functions that is used to get the 

2. get_balance_sheets
   Summary: Get balance sheets for a ticker and the specified limit and time period

3. get_cash_flow_statements
   Summary: Get cash flow statements for a ticker

4. calculator
   Summary: Calculator powered by SymPy for comprehensive mathematical operations.

5. python_repl
   Summary: Execute Python code in a REPL environment with interactive PTY support and state persistence.

6. file_read
   Summary: File reading tool with search capabilities, various reading modes, and document mode support for Bedrock compatibility.

7. mem0_memory
   Summary: Memory management tool for storing, retrieving, and managing memories in Mem0.

8. current_time
   Summary: Get the current time in ISO 8601 format.

9. journal
   Summary: Create and manage daily journal entries with tasks and notes



Strands Agents SDK provides support for asynchronous iterators through the stream_async method, enabling real-time streaming of agent responses in asynchronous environments like web servers, APIs, and other async applications.

In [25]:
# Define the async function
async def process_streaming_response(agent, query):
    agent_stream = agent.stream_async(query)
    async for event in agent_stream:
        # Print only the text content, not the raw event data
        if isinstance(event, dict):
            # Check for 'data' field first (seems to contain the text)
            if 'data' in event:
                print(event['data'], end='', flush=True)
            # Fallback to nested event structure
            elif 'event' in event and 'contentBlockDelta' in event['event']:
                delta = event['event']['contentBlockDelta']['delta']
                if 'text' in delta:
                    print(delta['text'], end='', flush=True)
        else:
            # If it's not a dict, just print it as is
            print(event, end='', flush=True)
    print()  # Add a newline at the end


In [26]:
import asyncio

# Define the query
query: str = """
I want you to perform a comprehensive financial analysis of AAPL. Please:

1. Retrieve and analyze the latest income statements, balance sheets, and cash flow statements for AAPL, 
if your tool encounters an error, then call the function again with the correct parameters.
2. Calculate key financial pointers from the data. Based on your output from the latency input statements, balance sheets and cash flow statements, 
generate code to create charts, pie charts, bar charts, execute mermaid diagrams to show interesting take aways from the financial data that you 
have in your context.
3. Identify trends over the past 3-5 years in revenue growth, profit margins, and cash generation
4. Assess Apple's financial health and liquidity position
5. Provide investment insights based on the financial data analysis
6. Store the key findings in memory for future reference
7. Create a journal entry summarizing today's AAPL analysis with actionable insights

Please present the analysis in a structured format with clear sections for each financial statement review, ratio analysis, trend identification, and final recommendations.
"""

In [ ]:
# simply run the agent to see the output 
finance_agent(query)

## Technical Analyst Agent
---

In this section, we will be crating the second sub agent that will be responsible for technical analysis to user questions. This inclues the following:

1. Questions about getting stock prices, i.e., based on a user question the ticker will go over a given data range and interval and the agent will get the stock prices.

2. Questions about seeking information on the current stock price - if the user asks for the current stock price, then the agent will call a tool to get the latest or current stock price for a given ticker.

3. Last, we will create a human in the loop for calculating technical indicators for a given time period.

In [ ]:
technical_agent_analysis_prompt: str = """
You are a technical analysis assistant that helps users analyze stock price movements and technical indicators. You can calculate and interpret various technical indicators using historical price data.

You have access to the following technical analysis capabilities based on the API key you have access to to call the available functions and get the following information:

1. Stock Price Data:
   - Current stock prices
   - Historical price data with various intervals
   - Custom date range analysis

2. Technical Indicators:
   - RSI
   - MACD
   - SMA
   - EMA
   - Bollinger Bands

You require the user to provide:
1. A stock ticker symbol
2. The type of technical indicator they want to analyze
3. Optionally: specific time periods or date ranges

Available functions:
1. get_current_stock_price: Retrieve latest stock price
2. get_stock_prices: Get historical price data for a specified period
3. get_technical_indicators: Calculate technical indicators for analysis

If you do not have access to the data that the user is asking for, do not make up an answer. Be completely accurate and only provide analysis based on the available technical indicators and price data.

Only answer questions related to technical analysis and price data based on the provided functions. If unsure, acknowledge limitations"""

In [ ]:
@tool 
def get_stock_prices(ticker:str, start_date: str, end_date:str, limit:int =5000) -> Union[Dict, str]:
    """ 
    Get historical stock prices for a ticker within a date range.count
    
    Args: 
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        start_date (str): Start date in YYYY-MM-DD format
        end_date (str): End date in YYYY-MM-DD format
        limit (int): Maximum number of price records to return (default:5000)

    Returns:
        Union[Dict, str]: JSON reponse containing historical price data or error message        
    """

    api_key = os.environ.get("FINANCIAL_DATASET_API")
    logger.info(f"API Key present: {'yes' if api_key else 'no'}")
    if not api_key:
        logger.error("Missing FINANCIAL_DATASET_API environment variable")
        return {"error": "Missing "}